# Save results

> Module die alle de resultaten van de `route_get` aanpast naar het gewenste format en aanvult met extra gegevens zoals de URL naar google maps.
> Deze functies hoeven in principe niet door de gebruiker zelf te worden aangeroepen.

In [ ]:
#| default_exp data_export

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from datetime import datetime
import os

import pandas as pd
import numpy as np
from fastcore.utils import L

from project.route_get import create_optimized_route
from project.utils import load_settings

In [ ]:
#| hide
# only used for testing while developping
settings = load_settings()
settings

{'files': {'path_peilbuizenshape_file': '/home/jelle/code/peilbuizen_route_optimalisatie/data/Grondwatermeetpunten/Grondwatermeetpunten_2024-10-10.shp',
  'path_pickle_results': '/home/jelle/code/peilbuizen_route_optimalisatie/data/pickles',
  'save_to_pickle': True,
  'path_results': '/home/jelle/results/peilbuizen_route_optimalisatie'},
 'filters': {'days_since_last_update': 200, 'projects': 'primair'},
 'calculation': {'distance_calculation_method': 'cycling-regular',
  'startlocation': 'Dokter van Deenweg 186, 8025 BM, Zwolle'},
 'azure': {'jdbc_hostname': 'wdodeuwprodsynoxgn02-ondemand.sql.azuresynapse.net',
  'jdbc_database': 'Lakehouse'},
 'sql_statement': {'tsid': "select [Site_Name], [Site_Longname], [Site_Number], SI.[STAS_ID], STA.[STA_ID], [Station_name], [Timeseries_name], [TS_ID], [WARECO_id]from [Datamart].[DIM_WDOD_STATIONS] ST, [Datamart].[DIM_WDOD_STATIONS_ATTRIBUTES] STA, [Datamart].[DIM_WDOD_SITES] SI, [Datamart].[DIM_WDOD_SITES_ATTRIBUTES] SIA, [Datamart].[DIM_WDOD

In [ ]:
#| hide
# Used to test
from fastcore.utils import Path
from project.data_get import load_pickle


In [ ]:
#| hide
# Used to test
file_path = settings['files']['path_pickle_results']
file_name = "get_data_from_azuresql_20250124_130111.pickle"
peilbuizen_df = load_pickle(file_path=Path(file_path) / file_name)

df_grouped = peilbuizen_df.groupby('project')
df_test = df_grouped.get_group('---')

In [ ]:
#| hide

settings['calculation']['startlocation']

'Dokter van Deenweg 186, 8025 BM, Zwolle'

In [ ]:
#| hide
optimal_route_test = create_optimized_route(start_address=settings['calculation']['startlocation'],
                                       route_profile=settings['calculation']['distance_calculation_method'],
                                       df=df_test)
optimal_route_test

{'type': 'FeatureCollection',
 'features': [{'bbox': [6.110889, 52.305248, 6.350065, 52.650295],
   'type': 'Feature',
   'properties': {'way_points': [0,
     147,
     284,
     291,
     351,
     572,
     950,
     968,
     1444,
     1560,
     2129],
    'summary': {'distance': 112416.5, 'duration': 24094.799999999996}},
   'geometry': {'coordinates': [[6.124844, 52.506756],
     [6.123812, 52.507825],
     [6.123066, 52.508671],
     [6.122329, 52.50961],
     [6.121859, 52.510382],
     [6.121788, 52.510596],
     [6.121671, 52.510801],
     [6.121609, 52.51091],
     [6.121549, 52.511013],
     [6.121358, 52.511228],
     [6.120941, 52.512188],
     [6.120844, 52.512609],
     [6.120531, 52.513566],
     [6.120515, 52.513764],
     [6.120491, 52.51383],
     [6.120462, 52.513916],
     [6.120391, 52.514115],
     [6.121654, 52.514187],
     [6.121754, 52.514193],
     [6.121764, 52.514139],
     [6.121768, 52.514107],
     [6.122169, 52.514114],
     [6.122854, 52.514155],
 

# Extract the order in which the travel locations (peilbuizen) are visited in the optimized route.

De Openrouteservice API geeft een json object terug. Dit object bevat een route met daarin alle punten die relevant zijn voor de routebeschrijving. Dus kruisingen, afslagen, etc. De functie `get_waypoint_coords` haalt uit deze totale lijst met punten de volgorde van de peilbuizen en geeft alleen deze peilbuizen en hun locaties terug. De functie maakt gebruik van het feit dat in de json van de Openrouteservice in de key 'way_points' is aangegeven op welke index in de totale routebeschrijving de punten staan waarvoor de route berekent is. Met andere woorden, voor de peilbuizen route, de 'way_points' key geeft aan op welke index in de totale routebeschrijving de peilbuizen staan.

In [ ]:
#| export

def get_waypoint_coords(route):
    waypoints_idx = route['features'][0]['properties']['way_points']
    return np.array(route['features'][0]['geometry']['coordinates'])[waypoints_idx]


In [ ]:
#| hide

waypoint_coords_test = get_waypoint_coords(optimal_route_test)
waypoint_coords_test

array([[ 6.124844, 52.506756],
       [ 6.21049 , 52.51439 ],
       [ 6.206965, 52.602474],
       [ 6.222663, 52.598154],
       [ 6.181068, 52.613973],
       [ 6.219549, 52.650245],
       [ 6.3491  , 52.501541],
       [ 6.322415, 52.500428],
       [ 6.203016, 52.306267],
       [ 6.179012, 52.359654],
       [ 6.124844, 52.506756]])

De functie `find_closest_point` gebruiken we om de coordinaten van de 'way_points' van de json route weer te koppelen aan de oorspronkelijke coordinaten van de peilbuizen in de dataframe. Dit is nodig, omdat de meeste peilbuizen niet op een openbare weg staan. De Openrouteservice API geeft in die gevallen de locatie van het punt op een openbare weg zo dicht mogelijk bij de peilbuis.

In [ ]:
#| export
def find_closest_point(target_lon, target_lat, df):
    distances = ((df['Longitude'] - target_lon)**2 + 
                (df['latitude'] - target_lat)**2)**0.5
    return distances.idxmin()


In [ ]:
#| export
def convert_routejson_to_df(waypoints_coords, df, remove_start=True, remove_end=True):
    """Order the input dataframe by the order of the waypoints in the route json given in the 
    route_coords parameter. If remove_start or remove_end are set to True, the first and/or last
    items from the route_coords are skipped. This makes it possible to account for a start location
    and or end location that don't have to be included in the returned route file.
    """
    start_idx = 1 if remove_start else 0
    end_idx = -1 if remove_end else None
    waypoints_coord = waypoints_coords[start_idx:end_idx]
    ordered_indices = list(map(lambda coord: find_closest_point(coord[0], coord[1], df), waypoints_coord))
    return df.loc[ordered_indices]


In [ ]:
#| hide
ordered_df_test = convert_routejson_to_df(waypoint_coords_test, df_test)
ordered_df_test

,Id,Longitude,latitude,latest_measure_date,project
511,21HG009A;400972,6.209705,52.513290,2024-07-17 12:05:53,---
296,21FC011A;394286,6.207653,52.601814,2024-04-09 09:51:57,---
587,21FC112A;394314,6.223631,52.598305,2024-04-09 10:03:06,---
389,B21E0010;416303,6.181055,52.613954,2024-04-09 10:43:36,---
451,B21F0003;416597,6.219551,52.650246,2024-04-09 10:57:31,---
593,22CC011A;403940,6.349587,52.500679,2024-04-09 12:31:02,---
957,21HC811B;400860,6.322404,52.499485,2024-04-09 11:47:35,---
242,27HC007B;410283,6.202980,52.306282,2024-11-04 11:26:39,---
241,27GC901A;408995,6.180674,52.359436,2024-04-03 10:55:02,---


Create the final route dataframe with:
- Added columns to the ordered dataframe to show the previous and next points on the route.
- Added columns to the ordered dataframe to show the URL to the location on Google Maps.

In [ ]:
#| export
def google_point_url(lat, lon):
    if np.isnan(lat) or np.isnan(lon):
        return None
    return f"https://www.google.com/maps?q={lat},{lon}"

In [ ]:
#| export
def create_route_prev_next(route: pd.DataFrame):
    ordered_df = route.drop(columns=["latest_measure_date"], axis=1)
    previous_df = ordered_df.shift(1)
    next_df = ordered_df.shift(-1)
    
    previous_df = previous_df.drop('project', axis=1)
    next_df = next_df.drop('project', axis=1)
    
    ordered_df['URL'] = ordered_df.apply(lambda row: google_point_url(row['latitude'], row['Longitude']), axis=1)
    previous_df['URL'] = previous_df.apply(lambda row: google_point_url(row['latitude'], row['Longitude']), axis=1)
    next_df['URL'] = next_df.apply(lambda row: google_point_url(row['latitude'], row['Longitude']), axis=1)
    
    previous_df = previous_df.rename(columns=lambda x: f'vorige_{x}')
    next_df = next_df.rename(columns=lambda x: f'volgende_{x}')
    
    return pd.concat([ordered_df, previous_df, next_df], axis=1)

In [ ]:
#| hide
route_prev_next_test = create_route_prev_next(ordered_df_test)
route_prev_next_test

,Id,Longitude,latitude,project,URL,vorige_Id,vorige_Longitude,vorige_latitude,vorige_URL,volgende_Id,volgende_Longitude,volgende_latitude,volgende_URL
511,21HG009A;400972,6.209705,52.513290,---,"https://www.google.com/maps?q=52.513289835589,...",None,NaN,NaN,None,21FC011A;394286,6.207653,52.601814,https://www.google.com/maps?q=52.6018140077626...
296,21FC011A;394286,6.207653,52.601814,---,https://www.google.com/maps?q=52.6018140077626...,21HG009A;400972,6.209705,52.513290,"https://www.google.com/maps?q=52.513289835589,...",21FC112A;394314,6.223631,52.598305,https://www.google.com/maps?q=52.5983052709003...
587,21FC112A;394314,6.223631,52.598305,---,https://www.google.com/maps?q=52.5983052709003...,21FC011A;394286,6.207653,52.601814,https://www.google.com/maps?q=52.6018140077626...,B21E0010;416303,6.181055,52.613954,https://www.google.com/maps?q=52.6139544769308...
389,B21E0010;416303,6.181055,52.613954,---,https://www.google.com/maps?q=52.6139544769308...,21FC112A;394314,6.223631,52.598305,https://www.google.com/maps?q=52.5983052709003...,B21F0003;416597,6.219551,52.650246,https://www.google.com/maps?q=52.6502456191275...
451,B21F0003;416597,6.219551,52.650246,---,https://www.google.com/maps?q=52.6502456191275...,B21E0010;416303,6.181055,52.613954,https://www.google.com/maps?q=52.6139544769308...,22CC011A;403940,6.349587,52.500679,"https://www.google.com/maps?q=52.50067890897,6..."
593,22CC011A;403940,6.349587,52.500679,---,"https://www.google.com/maps?q=52.50067890897,6...",B21F0003;416597,6.219551,52.650246,https://www.google.com/maps?q=52.6502456191275...,21HC811B;400860,6.322404,52.499485,https://www.google.com/maps?q=52.4994853087927...
957,21HC811B;400860,6.322404,52.499485,---,https://www.google.com/maps?q=52.4994853087927...,22CC011A;403940,6.349587,52.500679,"https://www.google.com/maps?q=52.50067890897,6...",27HC007B;410283,6.202980,52.306282,"https://www.google.com/maps?q=52.30628223108,6..."
242,27HC007B;410283,6.202980,52.306282,---,"https://www.google.com/maps?q=52.30628223108,6...",21HC811B;400860,6.322404,52.499485,https://www.google.com/maps?q=52.4994853087927...,27GC901A;408995,6.180674,52.359436,"https://www.google.com/maps?q=52.359436355642,..."
241,27GC901A;408995,6.180674,52.359436,---,"https://www.google.com/maps?q=52.359436355642,...",27HC007B;410283,6.202980,52.306282,"https://www.google.com/maps?q=52.30628223108,6...",None,NaN,NaN,None


In [ ]:
#| hide
route_prev_next_test['URL']

511    https://www.google.com/maps?q=52.513289835589,...
296    https://www.google.com/maps?q=52.6018140077626...
587    https://www.google.com/maps?q=52.5983052709003...
389    https://www.google.com/maps?q=52.6139544769308...
451    https://www.google.com/maps?q=52.6502456191275...
593    https://www.google.com/maps?q=52.50067890897,6...
957    https://www.google.com/maps?q=52.4994853087927...
242    https://www.google.com/maps?q=52.30628223108,6...
241    https://www.google.com/maps?q=52.359436355642,...
Name: URL, dtype: object

Create the google maps URL that shows the route in google maps.

In [ ]:
#| export
def google_maps_route_url(way_points_coords_lonlat):
    total_coords_latlon = [[lat, lon] for lon, lat in way_points_coords_lonlat]
    origin_g = total_coords_latlon[0]
    dest_g = total_coords_latlon[-1]
    wayp_g = total_coords_latlon[1:-1]
    
    origin_str = f"{origin_g[0]},{origin_g[1]}"
    dest_str = f"{dest_g[0]},{dest_g[1]}"
    waypoints_str = "|".join(f"{wp[0]},{wp[1]}" for wp in wayp_g)
    
    return f"https://www.google.com/maps/dir/?api=1&origin={origin_str}&destination={dest_str}&waypoints={waypoints_str}"

In [ ]:
#| hide

google_route_test = google_maps_route_url(waypoint_coords_test)
google_route_test

'https://www.google.com/maps/dir/?api=1&origin=52.506756,6.124844&destination=52.506756,6.124844&waypoints=52.51439,6.21049|52.602474,6.206965|52.598154,6.222663|52.613973,6.181068|52.650245,6.219549|52.501541,6.3491|52.500428,6.322415|52.306267,6.203016|52.359654,6.179012'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()